In [ ]:
#https://wiki.archlinux.org/index.php/Remastering_the_Install_ISO

set -k
yay -S --noconfirm arch-install-scripts squashfs-tools
isofn="manjaro-gnome-20.0.3-200606-linux56.iso"
mntdir="/mnt/archiso"
isodir="/run/media/$USER/SED/VBOX/ISOS"
isodir="/home/$USER/Documents"
isodir="/mnt/SED/VBOX/ISOS"
isofp="$isodir/$isofn"
customdir="/home/$USER/customiso"
dfs="desktopfs.sfs"

sudo mkdir -p $mntdir
sudo mkdir -p $customdir
#cp $isofp /home/$USER/Documents
sudo mount -t iso9660 -o loop $isofp $mntdir
cp -a $mntdir $customdir
#cd $customdir/arch/x86_64
cd $customdir/manjaro/x86_64
sudo rm -rf squashfs-root
sudo unsquashfs $dfs
sudo cp $customdir/boot/vmlinuz-x86_64 $customdir/manjaro/x86_64/squashfs-root/boot/vmlinuz-linux
sudo mkdir -p squashfs-root/{etc,proc,sys,dev/pts}
#sudo umount -l /proc
sudo mount --bind squashfs-root squashfs-root
sudo mount -t proc proc $customdir/manjaro/x86_64/squashfs-root/proc
#sudo mount -t proc none squashfs-root/proc
sudo mount -t sysfs none $customdir/manjaro/x86_64/squashfs-root/sys
sudo mount -o bind /dev squashfs-root/dev
sudo mount -o bind /dev/pts squashfs-root/dev/pts  ## important for pacman (for signature check)
sudo cp -L /etc/resolv.conf squashfs-root/etc  ## this is needed to use networking within the chroot
sudo arch-chroot squashfs-root /bin/bash

#https://forum.manjaro.org/t/virtualbox-wont-work-no-sbin-vboxconfig-file/96517/4
#chroot
pacman-key --init
pacman-key --populate archlinux
pacman -Syu --force archiso linux
nano /etc/mkinitcpio.conf
HOOKS="base udev memdisk archiso_shutdown archiso archiso_loop_mnt archiso_pxe_common archiso_pxe_nbd archiso_pxe_http archiso_pxe_nfs archiso_kms block filesystems keyboard"
mkinitcpio -p linux

#clear cache
LANG=C pacman -Sl | awk '/\[installed\]$/ {print $1 "/" $2 "-" $3}' > /pkglist.txt
pacman -Scc
exit

#if updating kernel
mv squashfs-root/boot/vmlinuz-linux $customdir/arch/boot/x86_64/vmlinuz
mv squashfs-root/boot/initramfs-linux.img $customdir/arch/boot/x86_64/archiso.img
rm squashfs-root/boot/initramfs-linux-fallback.img

mv squashfs-root/pkglist.txt ~/customiso/arch/pkglist.x86_64.txt

rm $dfs
mksquashfs squashfs-root $dfs

rm -r squashfs-root
sha512sum $dfs > airootfs.sha512

mkdir mnt
mount -t vfat -o loop ~/customiso/EFI/archiso/efiboot.img mnt
cp $customdir/arch/boot/x86_64/vmlinuz mnt/EFI/archiso/vmlinuz.efi
cp $customdir/arch/boot/x86_64/archiso.img mnt/EFI/archiso/archiso.img

dd if=/dev/zero bs=1M count=50 of=efiboot-new.img
mkfs.fat -n "ARCHISO_EFI" efiboot-new.img
mkdir new
mount -t fat -o loop efiboot-new.img new
cp -r mnt/* new/
umount new mnt
mv efiboot-new.img $customdir/EFI/archiso/efiboot.img

#ISO
genisoimage -l -r -J -V "ARCH_201209" -b isolinux/isolinux.bin -no-emul-boot -boot-load-size 4 -boot-info-table -c isolinux/boot.cat -o ../arch-custom.iso ./